In [2]:
import tensorflow as tf

### Using optimizer (using tf.Print)

In [14]:

# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

# Parameters
learning_rate = 0.01
training_epochs = 10
batch_size = 100
display_step = 1


# tf Graph Input
x = tf.placeholder(tf.float32, [None, 784]) # mnist data image of shape 28*28=784
y = tf.placeholder(tf.float32, [None, 10]) # 0-9 digits recognition => 10 classes

# Set model weights
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

# Construct model & Print intermediate data
## NOTE: tf.Print writes to stderr which does not show up in notebook. Rather the print statements
## can be checked in server where the notebook was started
matmul = tf.matmul(x, W) + b
printMatMul = tf.Print(matmul,[matmul[0][0]],"MatMul[0][0]:") # Print node
pred = tf.nn.softmax(printMatMul) # Softmax
printPred = tf.Print(pred,[tf.reduce_sum(pred)],"tf.reduce_sum(Pred):") # Print node

print (pred)

# Minimize error using cross entropy
cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(printPred), reduction_indices=1))
#cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred,labels=y))

optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

# Start training
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            # Fit training using batch data
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_xs,
                                                       y: batch_ys})
            
          
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if (epoch+1) % display_step == 0:
#             print(sess.run(W))
            print ("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))

    print ("Optimization Finished!")

    # Test model
    correct_prediction = tf.equal(tf.argmax(printPred, 1), tf.argmax(y, 1))
    # Calculate accuracy for 3000 examples
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print ("Accuracy:", accuracy.eval({x: mnist.test.images[:3000], y: mnist.test.labels[:3000]}))

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Tensor("Softmax_12:0", shape=(?, 10), dtype=float32)
('Epoch:', '0001', 'cost=', '1.183875584')
('Epoch:', '0002', 'cost=', '0.665347175')
('Epoch:', '0003', 'cost=', '0.552892696')
('Epoch:', '0004', 'cost=', '0.498688066')
('Epoch:', '0005', 'cost=', '0.465501576')
('Epoch:', '0006', 'cost=', '0.442650832')
('Epoch:', '0007', 'cost=', '0.425530611')
('Epoch:', '0008', 'cost=', '0.412152070')
('Epoch:', '0009', 'cost=', '0.401412889')
('Epoch:', '0010', 'cost=', '0.392407599')
Optimization Finished!
('Accuracy:', 0.87233335)


## MNIST using tf.gradient

Not using a default optimizer, rather, calculate gradients and update the weights manually

In [6]:

# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

# Parameters
learning_rate = 0.01
training_epochs = 10
batch_size = 100
display_step = 1

# Parameters
learning_rate = 0.01
training_epochs = 10
batch_size = 100
display_step = 1

# tf Graph Input
x = tf.placeholder(tf.float32, [None, 784]) # mnist data image of shape 28*28=784
y = tf.placeholder(tf.float32, [None, 10]) # 0-9 digits recognition => 10 classes

# Set model weights
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

# Construct model
pred = tf.nn.softmax(tf.matmul(x, W) + b) # Softmax

# Minimize error using cross entropy
cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(pred), reduction_indices=1))

# Calculating gradients
grad_W, grad_b = tf.gradients(xs=[W, b], ys=cost)
printGradW = tf.Print (grad_W,[tf.reduce_sum(grad_W)])
#printGradW = tf.Print (grad_b,[grad_b])

new_W = W.assign(W - learning_rate * printGradW)
new_b = b.assign(b - learning_rate * grad_b)

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

# Start training
with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            # Fit training using batch data
            _, _,  c = sess.run([new_W, new_b ,cost], feed_dict={x: batch_xs,
                                                       y: batch_ys})
            
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if (epoch+1) % display_step == 0:
#             print(sess.run(W))
            print ("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))

    print ("Optimization Finished!")

    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # Calculate accuracy for 3000 examples
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print ("Accuracy:", accuracy.eval({x: mnist.test.images[:3000], y: mnist.test.labels[:3000]}))

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
('Epoch:', '0001', 'cost=', '1.183969047')
('Epoch:', '0002', 'cost=', '0.665276117')
('Epoch:', '0003', 'cost=', '0.552849468')
('Epoch:', '0004', 'cost=', '0.498716198')
('Epoch:', '0005', 'cost=', '0.465515869')
('Epoch:', '0006', 'cost=', '0.442607517')
('Epoch:', '0007', 'cost=', '0.425555217')
('Epoch:', '0008', 'cost=', '0.412241959')
('Epoch:', '0009', 'cost=', '0.401435131')
('Epoch:', '0010', 'cost=', '0.392415389')
Optimization Finished!
('Accuracy:', 0.87266666)
